In [1]:
import torch
from torch import nn
from d2l import torch as d2l
import pandas as pd

In [2]:
def conv_block(input_channels, num_channels):
    return nn.Sequential(
        nn.BatchNorm2d(input_channels), nn.ReLU(),
        nn.Conv2d(input_channels, num_channels, kernel_size=3, padding=1))

In [3]:
def conv11_block(input_channels, num_channels):
    return nn.Sequential(
        nn.BatchNorm2d(input_channels), nn.ReLU(),
        nn.Conv2d(input_channels, num_channels, kernel_size=1))

In [4]:
def bottlenet_block(input_channels, num_channels):
    return nn.Sequential(conv11_block(input_channels, 4*num_channels),
            conv_block(4*num_channels, num_channels))

In [5]:
class DenseBlock(nn.Module):
    def __init__(self, num_convs, input_channels, num_channels):
        super(DenseBlock, self).__init__()
        layer = []
        for i in range(num_convs):
            layer.append(bottlenet_block(num_channels * i + input_channels, num_channels))
        self.net = nn.Sequential(*layer)

    def forward(self, X):
        for blk in self.net:
            Y = blk(X)
            # 连接通道维度上每个块的输入和输出
            X = torch.cat((X, Y), dim=1)
        return X

In [6]:
blk = DenseBlock(2, 3, 10)
X = torch.randn(4, 3, 8, 8)
Y = blk(X)
Y.shape

torch.Size([4, 23, 8, 8])

In [7]:
def transition_block(input_channels, num_channels):
    return nn.Sequential(
        nn.BatchNorm2d(input_channels), nn.ReLU(),
        nn.Conv2d(input_channels, num_channels, kernel_size=1),
        nn.AvgPool2d(kernel_size=2, stride=2))

In [8]:
b1 = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
    nn.BatchNorm2d(64), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [9]:
num_channels, growth_rate = 64, 32
num_convs_in_dense_blocks = [6, 12, 24, 16]

blks = []
for i, num_convs in enumerate(num_convs_in_dense_blocks):
    blks.append(DenseBlock(num_convs, num_channels, growth_rate))
    num_channels += num_convs * growth_rate
    if i != len(num_convs_in_dense_blocks) - 1:
        blks.append(transition_block(num_channels, num_channels // 2))
        num_channels = num_channels // 2

In [10]:
net = nn.Sequential(
    b1, *blks,
    nn.BatchNorm2d(num_channels), nn.ReLU(),
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Linear(num_channels, 1000))

In [11]:
X = torch.randn(1, 3, 224, 224)
for l in net:
    X = l(X)
    print(l.__class__.__name__, X.shape)

Sequential torch.Size([1, 64, 56, 56])
DenseBlock torch.Size([1, 256, 56, 56])
Sequential torch.Size([1, 128, 28, 28])
DenseBlock torch.Size([1, 512, 28, 28])
Sequential torch.Size([1, 256, 14, 14])
DenseBlock torch.Size([1, 1024, 14, 14])
Sequential torch.Size([1, 512, 7, 7])
DenseBlock torch.Size([1, 1024, 7, 7])
BatchNorm2d torch.Size([1, 1024, 7, 7])
ReLU torch.Size([1, 1024, 7, 7])
AdaptiveAvgPool2d torch.Size([1, 1024, 1, 1])
Flatten torch.Size([1, 1024])
Linear torch.Size([1, 1000])


In [12]:
rec = []
def ParamNum(l):
    num = 0
    expr = ''
    if isinstance(l, nn.Sequential):
        for l2 in l:
            num += ParamNum(l2)
    elif isinstance(l, DenseBlock):
        num = ParamNum(l.net)
    elif isinstance(l, nn.Conv2d):
        num = l.in_channels * l.out_channels * l.kernel_size[0] * l.kernel_size[1]
        expr = f'{l.in_channels}*{l.out_channels}*{l.kernel_size[0]}*{l.kernel_size[1]}'
    elif isinstance(l, nn.Linear):
        num = l.in_features * l.out_features
        expr = f'{l.in_features}*{l.out_features}'
        
    if expr != '':
        rec.append([l.__class__.__name__, expr, num])
    return num
sum_num = ParamNum(net)
print(f'{sum_num} = {round(sum_num / 1000 / 1000, 1)}M')
pd.DataFrame(rec)

7894208 = 7.9M


,0,1,2
0,Conv2d,3*64*7*7,9408
1,Conv2d,64*128*1*1,8192
2,Conv2d,128*32*3*3,36864
3,Conv2d,96*128*1*1,12288
4,Conv2d,128*32*3*3,36864
...,...,...,...
116,Conv2d,960*128*1*1,122880
117,Conv2d,128*32*3*3,36864
118,Conv2d,992*128*1*1,126976
119,Conv2d,128*32*3*3,36864
